### Innoviz assigment:

    Sign detection and classification by point cloud.
    Algorithm is tested on provided data and simuations.

    By Sergey Sinitsa

Module import:

In [ ]:
from pathlib import Path
import argparse

import numpy as np

import algorithms as alg
import io_utils as io


# Test on provided data

Data path specification:

In [ ]:
cluster_path = Path(r"Objects\cluster_2.csv")
min_ratio = 0.75


Data preview:

In [ ]:
points = io.read_csv(cluster_path)
scene = io.Scene_viewer(points)  # Initialization of scene view
sign_detector = alg.Bi_Modal()  # Initialization of bi modal detector

scene.show_cluster(points, False, title='Scene preview')
scene.show_cluster(points, True, title='Scaled scene preview')  # Cluster preview

Detection of Bi-Modal reflectivity density:

In [ ]:
sign_detector.fit_kde(points[:, -1])
dens, dens_x = sign_detector.produce_density_arr(points[:, -1], 100, show=False) # Generation of smooth density function
mode_status = sign_detector.detect_modes(dens_x, dens) # Mode detection and threshold detection

io.status_report(mode_status)  # Stops the execution if not a sign

Plate orientation detection:

In [ ]:
points_plate, points_pole = sign_detector.separate_by_thresh(points)  # Two sets of points - pole and plate

scene.show_cluster(points_plate, True)  # Plate preview
scene.show_cluster(points_pole, True)  # Pole preview

plate_plane = alg.Plate(min_ratio)  # At least 75% of points have to form a plane
plate_status = plate_plane.detect_plane_coefs(points_plate)  # Detect the 3D plane coefficients and normal

io.status_report(plate_status)  # Stops the execution if not a sign

Plate preview by inliers and outliers:

In [ ]:
scene.show_cluster(np.concatenate((plate_plane.inliers, plate_plane.outliers), 0),
                   True)  # Inliers and outliers preview

projected_points = plate_plane.project_to_plate(points_plate)  # Project plate points to plate, including outliers

Preview of points projection to plate plane:

In [ ]:
scene.show_cluster(projected_points, True)  # Preview the projected points

projected_points, img = plate_plane.rotate_plane(projected_points)  # Align plate plane with xy plane

Preview of points in image space:

In [ ]:
io.show_image(img, title='Points in pixel space')  # Preview the image

Shape detection:

In [ ]:
shape_type = plate_plane.detect_shapes(img)  # Check what shape suits the most to plate

print(f'The {shape_type} shaped sign is detected.')


